## Initialization

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter, OrderedDict
from dataclasses import dataclass
from pathlib import Path
import os
import pickle
import random
import math

import json

import numpy as np

from typing import Union, Optional
from types import MethodType

import matplotlib.pyplot as plt
import plotly.express as px
    
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

import torch
from torch.utils.data import TensorDataset, Dataset, IterableDataset, DataLoader, Sampler, BatchSampler
from torch.distributions import Dirichlet, Categorical
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import tqdm
import wandb

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from pathlib import Path
root_dir = Path("/mnt/data/factcheck/tweets/clustering/")

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## AverageMeter

In [2]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## MAB, SAB, ISAB, ...(code from openXAI and Jakub Monhart, edited)

In [3]:
class AddCompat(nn.Module):
    def __init__(self, size):
        super().__init__()
        self.v = nn.Parameter(torch.Tensor(size)) # importance vector
        bound = 1 / self.v.shape[0]**.5
        self.v.data.uniform_(-bound, bound)

    def forward(self, Q, K):
        Q = Q.unsqueeze(-2)        # (*, m, 1, H)
        K  = K.unsqueeze(-3)           # (*, 1, n, H)
        QK = torch.tanh(Q + K)              # (*, m, n, H)
        A_logits = torch.matmul(QK, self.v) # (*, m, n)
        return A_logits
  
    
class MultiCompat(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
  
    def forward(self, Q, K):
        A_logits = (Q @ K.transpose(1,2)) / math.sqrt(self.dim)
        return A_logits

    
class MAB(nn.Module):
    def __init__(self, dim_X, dim_Y, dim, num_heads=4, ln=False, p=None, compat='multi'):
        super().__init__()
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_X, dim)
        self.fc_k = nn.Linear(dim_Y, dim)
        self.fc_v = nn.Linear(dim_Y, dim)
        self.fc_o = nn.Linear(dim, dim)

        self.ln_preX = nn.LayerNorm(dim_X) if ln else nn.Identity()
        self.ln_preY = nn.LayerNorm(dim_Y) if ln else nn.Identity()
        
        # self.ln1 = nn.LayerNorm(dim) if ln else nn.Identity()
        # self.ln2 = nn.LayerNorm(dim) if ln else nn.Identity()
        self.ln1 = nn.Identity()  # no MAB here, we put it before ISAB
        self.ln2 = nn.Identity()
        
        self.dropout1 = nn.Dropout(p=p) if p is not None else nn.Identity()
        self.dropout2 = nn.Dropout(p=p) if p is not None else nn.Identity()
    
        if compat == 'multi':
            self.compat = MultiCompat(dim)
        elif compat == 'add':
            self.compat = AddCompat(dim//num_heads)
        else:
            print(f'Compatibility function {compat} not implemented.')
            return -1

    def forward(self, X, Y, mask=None):
        X = self.ln_preX(X)
        Y = self.ln_preY(Y)
        Q, K, V = self.fc_q(X), self.fc_k(Y), self.fc_v(Y)

        # print(Q.shape) # [1, 32, 256]
        # print(K.shape) # [1, 20, 256]
        # print(V.shape) # [1, 20, 256]
        Q_ = torch.cat(Q.chunk(self.num_heads, -1), 0)
        K_ = torch.cat(K.chunk(self.num_heads, -1), 0)
        V_ = torch.cat(V.chunk(self.num_heads, -1), 0)
        # print([e.shape for e in Q.chunk(self.num_heads, -1)]) # [[1, 32, 64],....
        # assert False

        # Compute compatibility
        A_logits = self.compat(Q_, K_)

        if mask is not None:
            mask = mask.squeeze(-1).unsqueeze(1)
            mask = mask.repeat(self.num_heads, Q.shape[1], 1)
            A_logits.masked_fill_(mask, -100.0)
        A = torch.softmax(A_logits, -1)

        attn = torch.cat((A @ V_).chunk(self.num_heads, 0), -1)
        O = self.ln1(Q + self.dropout1(attn))
        O = self.ln2(O + self.dropout2(F.relu(self.fc_o(O))))
        return O

    
class SAB(nn.Module):
    def __init__(self, dim_X, dim, **kwargs):
        super().__init__()
        self.mab = MAB(dim_X, dim_X, dim, **kwargs)

    def forward(self, X, mask=None):
        return self.mab(X, X, mask=mask)

    
class StackedSAB(nn.Module):
    def __init__(self, dim_X, dim, num_blocks, **kwargs):
        super().__init__()
        self.blocks = nn.ModuleList([SAB(dim_X, dim, **kwargs)])
        for i in range(num_blocks-1):
            self.blocks.append(SAB(dim, dim, **kwargs))
      
    def forward(self, X, mask=None):
        for sab in self.blocks:
            X = sab(X, mask=mask)
        return X

    
class PMA(nn.Module):
    '''
    If used as pooling:
    - num_inds: should be set to 1 for pooling
    - dim_x: dimension of input
    - dim: dimension of output
    '''
    def __init__(self, dim_X, dim, num_inds, **kwargs):
        super().__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim))
        nn.init.xavier_uniform_(self.I)
        self.mab = MAB(dim, dim_X, dim, **kwargs)

    def forward(self, X, mask=None):
        return self.mab(self.I.repeat(X.shape[0], 1, 1), X, mask=mask)
    
    
class ISAB(nn.Module):
    def __init__(self, dim_X, dim, num_inds, **kwargs):
        super().__init__()
        self.pma = PMA(dim_X, dim, num_inds, **kwargs)
        self.mab = MAB(dim_X, dim, dim, **kwargs)
        # we put pre-LN here
        self.ln = nn.LayerNorm(dim_X) if kwargs['ln'] else nn.Identity()

    def forward(self, X, mask=None):
        X = self.ln(X)
        return self.mab(X, self.pma(X, mask=mask))

    
class StackedISAB(nn.Module):
    def __init__(self, dim_X, dim, num_inds, num_blocks, **kwargs):
        super().__init__()
        self.blocks = nn.ModuleList([ISAB(dim_X, dim, num_inds, **kwargs)])
        for i in range(num_blocks-1):
            self.blocks.append(ISAB(dim, dim, num_inds, **kwargs))

    def forward(self, X, mask=None):
        for isab in self.blocks:
            # residual connectoin here
            X = X + isab(X, mask=mask)
        return X

## DAC Module (code from openXAI and Jakub Monhart, edited)

In [4]:
class DACModel(nn.Module):
    def __init__(self, args=None):
        super().__init__()
        a = args

        self.isab1 = StackedISAB(a.dim, a.dim, a.num_inds, a.num_blocks, 
                                p=a.drop_p, ln=a.ln, num_heads=a.num_heads)
        self.mab1 = MAB(a.input_size, a.input_size, a.dim, 
                       ln=a.ln, p=a.drop_p, num_heads=a.num_heads)
        self.mab2 = MAB(a.dim, a.dim, a.dim, 
                       ln=a.ln, p=a.drop_p, num_heads=a.num_heads)
        self.pma = PMA(a.dim, a.dim, 1,
                      ln=a.ln, p=a.drop_p, num_heads=a.num_heads)
        self.isab2 = StackedISAB(a.dim, a.dim, a.num_inds, a.num_blocks, 
                                 p=a.drop_p, ln=a.ln, num_heads=a.num_heads)
        self.fc = nn.Linear(a.dim, 1)


    def forward(self, X, anchor_idxs, mask=None):        
        xa = X[torch.arange(X.shape[0]), anchor_idxs].unsqueeze(-2)
        H_Xa = self.isab1(self.mab1(X, xa), mask=mask)

        # extract params of clusters including anchors
        H_theta = self.pma(H_Xa, mask=mask)

        # extract membership vector logits
        H_m = self.mab2(H_Xa, H_theta)
        H_m = self.isab2(H_m, mask=mask)
        logits = self.fc(H_m) # last dimension is squeezed in the calling PL DAC module
        
        return logits
    
        # variant B
        # xa = X[torch.arange(X.shape[0]), anchor_idxs].unsqueeze(1)
        # Xa = self.mab(X, xa)
        # H_Xa = self.isab(Xa, mask=mask)
        # x = self.fc(Xa)
        
        # variant C
        # H_enc = self.isab(X, mask=mask)
        # anchors = H_enc[torch.arange(X.shape[0]), anchor_idxs].unsqueeze(1)
        # H_enc = self.mab(H_enc, anchors)
        # x = self.fc(self.isab2(H_enc, mask=mask))

## DAC Lightning Module (code by Jan Drchal, updated)

In [5]:
class DACLightning(pl.LightningModule):
    def __init__(self, args, schedule_fn=None, featuremodel=None, bivalidate=True):
        super().__init__()
        self.args = args
        if featuremodel is None:
            featuremodel = nn.Identity()
        self.schedule_fn = schedule_fn
        self.featuremodel = featuremodel
        self.dacmodel = DACModel(args)

        self.max_class = args.max_class
        
        self.criterion = self.anchored_loss
        
        self.bivalidate = bivalidate

        self.save_hyperparameters("args")

    def sample_anchor_idxs(self, B, N, mask=None):
        # uniform distribution over bag elements, TODO try uniform over clusters
        if mask is None:
            # return torch.zeros([B], dtype=torch.int64)
            idxs = torch.randint(N, (B,))
            return idxs
        else:
            mask = mask.view(B, N)
            anchor_idxs = torch.zeros(B, dtype=torch.int64)
            for b in range(B):
                if mask[b].sum() < N:
                    idx_pool = mask[b].bitwise_not().nonzero().view(-1)
                    anchor_idxs[b] = idx_pool[torch.randint(len(idx_pool), [1])]
            return anchor_idxs

    def sample_anchor_idxs_partial(self, T):
        B, N = T.shape[:2]

        idxs = torch.zeros(B, dtype=torch.long)
        for b in range(B):
            # print(f"DACLightning.anchored_loss: T[b] = {T[b]}")
            cls_ids = T[b].unique(sorted=True)
            if cls_ids[0] != 0:
                cls_ids = torch.cat((torch.Tensor([0.]).to(self.device), cls_ids))
            # print(f"DACLightning.anchored_loss: cls_ids = {cls_ids}")
            last_cls_idx = self.max_class
            # print(f"DACLightning.anchored_loss: last_cls_id = {last_cls_idx}")
            
            # here changed, zero is a non-class
            if last_cls_idx > 1:
                cls_id = cls_ids[torch.randint(1, last_cls_idx+1, (1,)).item()]
            else:
                cls_id = cls_ids[1]
            assert cls_id in (1,2)
            if self.max_class == 1:
                assert cls_id == 1
                
            # print(f"DACLightning.anchored_loss: cls_id = {cls_id}")
            all_idxs = (T[b] == cls_id).nonzero()
            # print(f"DACLightning.anchored_loss: all_idxs = {all_idxs}")
            sel_idx = all_idxs[torch.randint(len(all_idxs), (1, )).item()].item()    
            idxs[b] = sel_idx
        return idxs


    def anchored_loss(self, logits, anchor_idxs, T):
        # own implementation instead of dac.anchored_cluster_loss
        # logits (batch_size, bag_size)
        # T      (batch_size, bag_size)
        batch_size, bag_size = logits.shape[0:2]
        # print(f"DACLightning.anchored_loss: logits.shape = {logits.shape}")
        anchor_cids = T[torch.arange(batch_size), anchor_idxs] 

        # same shape as logits and T: 1.0 for selected cluster, 0.0 otherwise
        targets = (T == anchor_cids.unsqueeze(-1)).float()

        loss = F.binary_cross_entropy_with_logits(logits, targets)
        return loss

    def forward(self, X, anchor_idxs, mask=None):
        # (batch_size, bag_size, input_dim_1, ....)
        batch_size, bag_size = X.shape[0:2]
        
        # Xre = X.reshape([batch_size*bag_size] + list(X.shape[2:])) # combine all bags in batch for the feature extraction phase
        # H = self.featuremodel(Xre)  # (batch_size*bag_size, feature_size)
        # H = H.reshape(batch_size, bag_size, -1) # get back batches and bags in them: (batch_size, bag_size, feature_size)
        
        H = X  # (batch_size, bag_size, feature_size)

        if mask is not None:
            # mask = (batch_size, bag_size)
            mask = mask.unsqueeze(-1) # this is shape accepted by dac.py: (batch_size, bag_size, 1)
        logits = self.dacmodel(H, anchor_idxs, mask=mask) 
        logits = logits.squeeze(-1) # (batch_size, bag_size)
        return logits

    def forward_loss(self, batch):
        # X.shape = (batch_size, bag_size, input_dim_1), T.shape = (batch_size, bag_size)
        X, T = batch
        batch_size, bag_size = X.shape[0], X.shape[1]
        if self.max_class is not None:
            anchor_idxs = self.sample_anchor_idxs_partial(T)
        else:
            assert False
            anchor_idxs = self.sample_anchor_idxs(batch_size, bag_size)

        logits = self(X, anchor_idxs)
        loss = self.criterion(logits, anchor_idxs, T)
        
        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward_loss(batch)
        self.log("loss", loss, prog_bar=True,
                 logger=True, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        # print("validation_step")
        loss, logits = self.forward_loss(batch)
        X, T = batch
        
        m = self.max_class
        self.max_class = 1
        plabels = self.cluster_anchored(X, T=T)
        self.max_class = m
        
        return {"loss": loss, "plabels": plabels, "targets": T}

    def predict_step(self, batch, batch_idx):
        # loss, logits = self.forward_loss(batch)
        X, T = batch
        
        m = self.max_class
        self.max_class = 1
        plabels = self.cluster_anchored(X, T=T)
        self.max_class = m
        
        return plabels

    def cluster_anchored(self, X, T=None, given_idxs=None, max_iter=1):        
        # print(f"DACLightning.cluster_anchored: X = {X.shape}")
        batch_size, bag_size = X.shape[0], X.shape[1]
        with torch.no_grad():
            mask = torch.zeros(batch_size, bag_size, dtype=torch.bool, device=self.device) # True marks already selected elements
            labels = torch.zeros(batch_size, bag_size, dtype=torch.long, device=self.device) 
            for i in range(1, max_iter + 1):
                # print(f"=============================================================")
                # print(f"DACLightning.cluster_anchored: i = {i}")

                if given_idxs is not None:
                    anchor_idxs = given_idxs
                elif T is not None:
                    anchor_idxs = self.sample_anchor_idxs_partial(T)
                else:
                    anchor_idxs = self.sample_anchor_idxs(batch_size, bag_size, mask=mask)
                # print(f"DACLightning.cluster_anchored: anchor_idx = {anchor_idxs}")
                
                logits = self.forward(X, anchor_idxs, mask)
                # print(f"DACLightning.cluster_anchored: logits = {logits.shape}")

                # logits positive => sigmoid > 0.5
                select = logits > 0.0
                
                # we can log during training only
                # not during inference by directly calling cluster_anchored
                # and giving it the anchor index
                if given_idxs is None:
                    self.log("lmin", logits.min(), prog_bar=True, logger=True, on_step=True, on_epoch=True)
                    self.log("lmax", logits.max(), prog_bar=True, logger=True, on_step=True, on_epoch=True)
                    # print(f"DACLightning.cluster_anchored: select = {select}")

                labels[select] = i
                mask[select] = True
                
                # print(f"DACLightning.cluster_anchored: labels = {labels}")
                # print(f"DACLightning.cluster_anchored: mask = {mask}")

                num_processed = mask.sum(1)
                done = num_processed == bag_size
                if done.all():
                    break

                # print(f"DACLightning.cluster_anchored: num_processed = {num_processed}")

                # print(f"DACLightning.cluster_anchored: done = {done}")
            # print(f"DACLightning.cluster_anchored: FINAL labels = {labels}")
            return labels

    def validation_epoch_end(self, validation_step_outputs):
        losses = AverageMeter()
        ari = AverageMeter()
        nmi = AverageMeter()
        nclusters = AverageMeter()
        
        def fn(vso, name, update_loss):
            for out in vso:
                if update_loss:
                    losses.update(out["loss"])
                targets = out["targets"].cpu()
                plabels = out["plabels"].cpu()
                targets[targets != 1] = 0  # targets are 1 (in_set) / 2 (out_set) valued, plabels are 1 (in_set) / 0 (out_set)
                val_acc = (targets==plabels).float().mean().item()
                val_true = plabels.float().mean().item()
                self.log(f"{name}_acc", val_acc, prog_bar=True, logger=True, on_step=False, on_epoch=True)
                self.log(f"{name}_pred", val_true, prog_bar=True, logger=True, on_step=False, on_epoch=True)

        if self.bivalidate:
            fn(validation_step_outputs[0], 'val', True)
            fn(validation_step_outputs[1], 'train', False)
        else:
            fn(validation_step_outputs, 'val', True)


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.args.lr)
        if self.schedule_fn is None:
            return optimizer
        else:
            scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, self.schedule_fn)
            return [optimizer], [scheduler]

## Loaders

In [6]:
import json
from pathlib import Path
import numpy as np

data_dir = Path('/mnt/data/factcheck/tweets/similarity_web_data')
annotations_path = Path('/mnt/data/factcheck/tweets/similarity_web_annotations')

vecs = torch.load(data_dir / 'all_tensors_pca.pt')
vecs /= vecs.norm(dim=1)[:, None]
print(np.linalg.norm(vecs, axis=1).mean(), np.linalg.norm(vecs, axis=1).std())
print(vecs.shape)

def load_dict(flnm):
    with open(flnm, 'rt') as f:
        data = json.load(f)
    data = {int(k): v for k, v in data.items()}
    return data

all_to_text = load_dict(data_dir / 'all_text.json')
all_to_vecnos = load_dict(data_dir / 'all_vecno.json')
claims = list(all_to_text.values())

anno_path = Path('/mnt/data/factcheck/tweets/similarity_web_annotations')
paths = [f for f in anno_path.iterdir() if f.is_file() and f.suffix == '.json']
paths = sorted(paths)

description, included, excluded = [], [], []
for path in paths:
    with path.open('rt') as f:
        data = json.load(f)
        d, i, e = data['description'], data['included'], data['excluded']
        
        description.append(d)
        
        vecnos = torch.tensor([all_to_vecnos[int(ii)] for ii in i])
        vecnos.sort()
        included.append(vecnos)
        
        
        vecnos = torch.tensor([all_to_vecnos[int(ee)] for ee in e])
        vecnos.sort()
        excluded.append(vecnos)
        
print(len(description), len(included), len(excluded))

# filter out 3 largest (we will have 45 and 17 then)

ilens = torch.tensor([len(i) for i in included])
upper_excl = ilens.sort()[0][-3].cpu().item()

vecnos = [i for i in included if len(i) < upper_excl]

permuter = torch.randperm(len(vecnos))

print('permutation', permuter)

shuffled_vecnos = [vecnos[i] for i in permuter]

train_split = shuffled_vecnos[:45]
val_split = shuffled_vecnos[45:]

len(train_split), len(val_split)

1.0 2.5724927e-07
torch.Size([203057, 512])
65 65 65
permutation tensor([52, 16, 27, 14, 48,  1, 32, 19, 28, 10, 30, 17, 29, 58, 11, 34, 44, 50,
        22,  2, 38, 15, 51, 13, 18,  9, 33, 37,  4, 20,  6, 39, 55, 21, 60,  0,
        61, 35, 40, 45, 54, 56, 24, 43, 36, 49, 26, 41, 53,  5, 23, 25,  7, 31,
        46,  3, 57, 42, 59, 47, 12,  8])


(45, 17)

In [7]:
CLUSTERS = 15000

out_path = Path('/mnt/data/factcheck/tweets/clustering/indices')

with open(out_path / f'kmeans_b0_{CLUSTERS}.npy', 'rb') as f:
    clustered_indices_all = np.load(f, allow_pickle=False)

kmeans_clusters = [(clustered_indices_all==group_no).nonzero()[0] 
                   for group_no in range(CLUSTERS)]

kmeans_clusters = [torch.tensor(g) for g in kmeans_clusters if len(g) > 5]

In [8]:
@torch.no_grad()
def cosine_choice(incl, k, vectors):
    """
    :param: incl: indices of vectors from vectors to look around from
    :param: k: number of closest vectors (including incl vectors) to be returned
    :param: vectors: dataset with all vectors (l x dim)
    :return: indices from vectors of k closest vectors (inluding those from incl)
    """
    res = None
    batch_size = 3500000 // len(incl)
    batches = math.ceil(len(vectors) / batch_size)
    
    res = []
    for i in range(batches):
        lo = i * batch_size
        hi = lo + batch_size  # exclusive
        vecs_batch = vectors[lo:hi, :]  # n x 512
        vecs_resh = vecs_batch[:, None, :]  # n x 1 x 512

        def get_multiplied(indices):
            sel = vectors[indices, :]  # k x 512
            sel_resh = sel.T[None, :, :]  # -(T)-> 512 x k -[]-> 1 x 512 x k

            mult = vecs_resh.to(device) @ sel_resh.to(device)
            # n x 1 x 512 @ 1 x 512 x k == n x 1 x k

            return mult.squeeze(dim=1)  # n x k

        mult = get_multiplied(incl)             # n x k

        mult = mult.mean(dim=1, keepdims=True)  # n x k -> n x 1
        res.append(mult)
        
    res = torch.cat(res, dim=0)
    res = res.squeeze(dim=1)
    val, indices = res.topk(k)
    return indices

In [9]:
def subtract_set(main, subtract):
    """
    subtracts the subract 'set' (a torch tensor) from the main 'set' (a torch tensor)
    and returns a torch tensor
    """
    return torch.from_numpy(
        np.setdiff1d(main.cpu().numpy(), subtract.cpu().numpy())
    )

def choose_random(in_tensors, num):
    """
    returns num randomly sampled tensors (num x d) from in_tensors (l x d)
    """
    permuter = torch.randperm(len(in_tensors))[:num]
    return in_tensors[permuter]

class DACDataset:
    
    def __init__(self, class_ids, tensors, bags, max_class_size, max_class_fraction,
                 bag_size, multiplier, method: str, seed=42):
        self.class_ids: list[torch.Tensor] = class_ids
        self.tensors: torch.Tensor = tensors
        self.bags: int = bags
        self.max_class_size: int = max_class_size
        self.max_class_fraction: float = max_class_fraction
        self.bag_size: int = bag_size
        self.multiplier = multiplier
        self.seed: int = seed
        self.method: str = method
        assert method in ('rrrn', 'cnn', 'ccn', 'cn', 't')
        # r: random ball, n: noise, c: class ball
        
    # class ball 1x, noise 2x
    def cnn(self, multip=2):
        me_bag, me_l = self.give_bags()
        rnd_bag, rnd_l = self.give_random_ball(multip * self.bag_size)
        
        rnd_l[:, :] = 2

        ids = torch.cat((me_bag, rnd_bag), dim=1)  # batch x (3*bag_size) x feature
        labels = torch.cat((me_l, rnd_l), dim=1)
        
        self.permute_(ids, labels)
        
        yield ids, labels
        
    # class ball 1x, noise 1x
    def cn(self):
        yield from self.cnn(multip=1)
    
    # class ball 1x, noise 2x
    def t(self):
        ids, labels = self.give_true()        
        self.permute_(ids, labels)
        yield ids, labels
    
    # class ball 2x, noise 1x
    def ccn(self):
        me_bag, me_l = self.give_bags()
        other_bag, other_l = self.give_bags()
        rnd_bag, rnd_l = self.give_random_ball(self.bag_size)
        
        print(me_bag.shape, other_bag.shape, rnd_bag.shape)
        
        other_l[:, :] = 2  # all items from other and random are out of the cluster
        rnd_l[:, :] = 2

        ids = torch.cat((me_bag, other_bag, rnd_bag), dim=1)  # batch x (3*bag_size) x feature
        labels = torch.cat((me_l, other_l, rnd_l), dim=1)
        
        self.permute_(ids, labels)
        
        yield ids, labels
        
    # random ball 3x, noise 1x
    def rrrn(self):
        bag1, _ = self.give_random_ball(80)
        ok = torch.ones((self.bags, 80), device=device)
        
        bag2, ko2 = self.give_random_ball(100)
        
        bag3, ko3 = self.give_random_ball(100)
        
        ran = self.give_random_noise(768 - 100 - 100 - 80)
        ko4 = 2 * torch.ones((self.bags, 768 - 100 - 100 - 80), device=device)
        
        ids = torch.cat((bag1, bag2, bag3, ran), dim=1) 
        labels = torch.cat((ok, ko2, ko3, ko4), dim=1)
        
        self.permute_(ids, labels)
        
        yield ids, labels
        
    def __iter__(self):
        method = getattr(self, self.method)  # self.method: string, method: bound fn
        yield from method()
           
    @torch.no_grad()
    def permute_(self, ids, labels):
        # bags, bag_len, features  bags, bag_len
        assert ids.shape[:2] == labels.shape[:2]
        bag_len = ids.shape[1]
        for i in range(len(ids)):
            permuter = torch.randperm(bag_len, device=device)
            ids[i, :, :] = ids[i, permuter, :]
            labels[i, :] = labels[i, permuter]
        
    @torch.no_grad()
    def give_random_noise(self, noise_size):
        indices = torch.randint(low=0, high=len(self.tensors), size=(self.bags, noise_size))
        tens = self.tensors[indices]
        # print(tens.shape)
        return tens
        
    @torch.no_grad()
    def give_random_ball(self, ball_size):
        which = torch.randint(low=0, high=len(self.tensors), size=(self.bags,))
        bags, labels = [], []
        for w in which:
            idx_around = cosine_choice(w.reshape(1), ball_size, self.tensors)
            ball_around = self.tensors[idx_around]
            bags.append(ball_around)
            
            lab = 2 * torch.ones(ball_size, dtype=int, device=device)
            labels.append(lab)
        bags, labels = torch.stack(bags, dim=0), torch.stack(labels, dim=0)
        return bags, labels
    
    @torch.no_grad()
    def give_true(self):
        which = torch.randint(low=0, high=len(self.class_ids), size=(self.bags,))
        bags, labels = [], []
        for w in which:
            w = w.item()
            good = self.class_ids[w][:self.max_class_size].to(device)
            assert len(good) > 0, good
            
            def make_bag(mult):
                closest = cosine_choice(good, int(self.bag_size * mult), self.tensors)
                closest_other = subtract_set(closest, self.class_ids[w]).to(device)
                
                # here we choose the points closest to the cluster
                missing_length = self.bag_size - len(good)
                chosen_fringe = closest_other[:missing_length]
                return torch.cat((good, chosen_fringe), dim=0)
            
            def make_large_enough_bag():
                current_multiplier = self.multiplier
                while True:
                    bag = make_bag(current_multiplier)
                    if len(bag) < self.bag_size:
                        current_multiplier += 1
                    else:
                        return bag
                    
            bag = make_large_enough_bag()
            ones = torch.ones(len(good), dtype=int, device=device)
            zeros = torch.zeros(len(bag) - len(good), dtype=int, device=device)
            lab = torch.cat((ones, zeros+2), dim=0) # added here provisionally so that we have
                                                    # class 1 (good) and 2 (bad)

            bags.append(self.tensors[bag])
            labels.append(lab)
        tensors, labels = torch.stack(bags, dim=0), torch.stack(labels, dim=0)
        return tensors, labels

    @torch.no_grad()
    def give_bags(self):
        which = torch.randint(low=0, high=len(self.class_ids), size=(self.bags,))
        bags, labels = [], []
        for w in which:
            w = w.item()
            max_size = min(int(self.max_class_fraction * len(self.class_ids[w])), 
                           self.max_class_size)
            good = choose_random(self.class_ids[w], num=max_size).to(device)
            assert len(good) > 0, good
            
            def make_bag(mult):
                closest = cosine_choice(good, int(self.bag_size * mult), self.tensors)
                closest_other = subtract_set(closest, self.class_ids[w]).to(device)
                
                # take 3 * more than we need, choose points further from the cluster
                missing_length = self.bag_size - len(good)
                outer_fringe = closest_other[-3*missing_length:]
                chosen_fringe = choose_random(outer_fringe, missing_length)
                return torch.cat((good, chosen_fringe), dim=0)
            
            def make_large_enough_bag():
                current_multiplier = self.multiplier
                while True:
                    bag = make_bag(current_multiplier)
                    if len(bag) < self.bag_size:
                        current_multiplier += 1
                    else:
                        return bag
                    
            bag = make_large_enough_bag()
            ones = torch.ones(len(good), dtype=int, device=device)
            zeros = torch.zeros(len(bag) - len(good), dtype=int, device=device)
            lab = torch.cat((ones, zeros+2), dim=0) # added here provisionally so that we have
                                                    # class 1 (good) and 2 (bad)

            bags.append(self.tensors[bag])
            labels.append(lab)
        tensors, labels = torch.stack(bags, dim=0), torch.stack(labels, dim=0)
        return tensors, labels
        
    def __len__(self):
        return 1
    
    def __getitem__(self, idx):
        return next(iter(self))
    
vecs = vecs.to(device)


# ds_rrrn = DACDataset(class_ids=None, method='rrrn', **params)
ds_train = DACDataset(tensors=vecs,
                          class_ids=kmeans_clusters, 
                          method='t', 
                          max_class_size=400,
                          bags=64,
                          max_class_fraction=1,
                          bag_size=800,
                          multiplier=4)

ds_val = DACDataset(tensors=vecs,
                        class_ids=shuffled_vecnos, 
                        method='t', 
                        max_class_size=400,
                        bags=64,
                        max_class_fraction=1,
                        bag_size=800,
                        multiplier=4)
# ds_annot30_ccn = DACDataset(class_ids=class_tensors[:30], method='ccn', **params)
# ds_annot45_cnn = DACDataset(class_ids=class_tensors, method='cnn', **params)     
# ds_kmeans_cnn = DACDataset(class_ids=kmeans_clusters, method='cnn', **params)

In [10]:
class Clusters:
    def __init__(self, clusters, anchor_idxs=None):
        self.clusters = [c.to(device) for c in clusters]
        self.maxsize = 800
        self.anchor_idxs = anchor_idxs
    
    def __len__(self):
        return len(self.clusters)
    
    def __iter__(self):
        return self
        
    def __getitem__(self, i):
        cluster = self.clusters[i]
        idx = torch.randint(low=0, high=len(cluster), size=(1,))
        which = cluster[idx].reshape(1)
        closest_indices = cosine_choice(which, min(self.maxsize, len(vecs)), vecs).to(device)
        closest_vectors = vecs[closest_indices].to(device)
        target = torch.zeros(self.maxsize, device=device) + 2
        for i, c in enumerate(closest_indices):
            if c in cluster:
                target[i] = 1
        return closest_vectors, target
            
clusters = Clusters(kmeans_clusters)

In [11]:
from torch.utils.data import DataLoader

dl_clusters = DataLoader(clusters, batch_size=40, shuffle=True)

In [12]:
a, b = next(iter(dl_clusters))
a.shape, b.shape

(torch.Size([40, 800, 512]), torch.Size([40, 800]))

## Train

In [13]:
def make_schedule_fn(warmup_epochs, stable_epochs, decreasing_epochs):
    def fn(epoch):
        if epoch < warmup_epochs:
            return epoch/warmup_epochs
        elif epoch < warmup_epochs + stable_epochs:
            return 1
        else:
            now_epoch = epoch - stable_epochs - warmup_epochs
            return 1 - now_epoch / decreasing_epochs
    return fn

# Cluster

In [14]:
import json
from pathlib import Path

anno_path = Path('/mnt/data/factcheck/tweets/similarity_web_annotations')
paths = [f for f in anno_path.iterdir() if f.is_file() and f.suffix == '.json']

annotated_groups = []
for path in paths:
    with path.open('rt') as f:
        included = json.load(f)['included']
        vecnos = np.array([all_to_vecnos[int(i)] for i in included])
        vecnos.sort()
        annotated_groups.append(vecnos)

In [15]:
out_path = Path('/mnt/data/factcheck/tweets/clustering/indices')

def intersection_count(ag: np.ndarray, clu: np.ndarray) -> int:
    return len(np.intersect1d(ag, clu, assume_unique=True))
               
def union_count(ag: np.ndarray, clu: np.ndarray) -> int:
    c = np.concatenate((ag, clu), axis=0)
    return len(np.unique(c))

def compute_ab(clustered_indices_all):

    clustered_groups = []
    for i in range(clustered_indices_all.max()):
        indices = (clustered_indices_all==i).nonzero()[0]
        indices.sort()
        clustered_groups.append(indices)
        
    a_to_c_inters = np.zeros((len(annotated_groups), len(clustered_groups)), dtype=int)
    a_to_c_union = np.zeros((len(annotated_groups), len(clustered_groups)), dtype=int)
    for a_idx, ag in enumerate(annotated_groups):
        inters = np.array([intersection_count(ag, clu) for clu in clustered_groups])
        a_to_c_inters[a_idx, :] = inters
        
        union = np.array([union_count(ag, clu) for clu in clustered_groups])
        a_to_c_union[a_idx, :] = union
    
    iou = a_to_c_inters / a_to_c_union

    iou_max = iou.max(axis=1)
    iou_amax = iou.argmax(axis=1)
    
    return iou_max.mean()

In [16]:
@dataclass
class ClusterArgs:
    model_type: str = 'dac'

    lr: float = 3e-5
    
    max_class: int = 2

    feature_size: int = 512  # number of input features
    dim: int = 512  # we reduce dimensionality
    num_blocks: int = 4  # Number of stacks of SABs/ISABs
    num_heads: int = 8
    
    num_inds: int = 16
    input_size: int = 512

    drop_p: float = 0.2  # Dropout
    ln: bool = True  # Layer normalization
    
args = ClusterArgs()


NAME = 'pretrain_c1_512l'
warm, main, post = 10, 90, 10
schedule_fn = make_schedule_fn(warm, main, post)

dacmodel = DACLightning(args=args, schedule_fn=schedule_fn)
dacmodel = DACLightning(args=args).load_from_checkpoint(root_dir / 'saved_models' / NAME, 
                                                        args=args, schedule_fn=schedule_fn)

logdir = "/mnt/data/factcheck/tweets/clustering/tb_logs"
lr_monitor = LearningRateMonitor(logging_interval='step')
# wandb.init(project="amortized_omniglot", name='omniglot400')
# wandb_logger = WandbLogger()


NAME = 'kmeans512_clustersX'

logger = TensorBoardLogger(logdir, name=NAME)

trainer = pl.Trainer(gpus=1, log_every_n_steps=10, max_epochs=warm+main+post-1,
                     default_root_dir=root_dir, check_val_every_n_epoch=1,
                     logger=logger,
                     callbacks=[lr_monitor]
                    )

trainer.fit(dacmodel, dl_clusters, val_dataloaders=[ds_val, 
                                                    dl_clusters])

trainer.save_checkpoint(root_dir / 'saved_models' / NAME)

Multiprocessing is handled by SLURM.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type     | Params
------------------------------------------
0 | featuremodel | Identity | 0     
1 | dacmodel     | DACModel | 20.1 M
------------------------------------------
20.1 M    Trainable params
0         Non-trainable params
20.1 M    Total params
80.302    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 1, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/petroja5/venv_amd/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is th

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [17]:
MAXSIZE = 800
dacmodel = dacmodel.to(device)


s = 0
for i, k in enumerate(kmeans_clusters):
    for which in k[:1]:
        which = which.to(device).reshape(1)
        closest_indices = cosine_choice(which, min(MAXSIZE, len(vecs)), vecs).to(device)
        anchor = (which==closest_indices).nonzero()[0].to(device)
        closest_vectors = vecs[closest_indices][None, :, :].to(device)
        res = dacmodel.cluster_anchored(closest_vectors, given_idxs=anchor).squeeze(0)
        
        target = torch.zeros(MAXSIZE, device=device)
        for i, c in enumerate(closest_indices):
            if c in k.to(device):
                target[i] = 1
        
        s += res.sum()
        # print(target.sum(), res.sum(), (res==target).sum())

In [ ]:
# vybrat 5 nejbližších bodů a ensemblovat
# střídavě posílat do batche vektor z kmeans a z trainsetu
# every point is from some cluster

dacmodel = dacmodel.to(device)

MAX_SIZES = [50, 100, 200, 500, 1000]

results, clusters = [], []
for MAXSIZE in MAX_SIZES:
    allvecs_indices = torch.arange(len(vecs), device=device, dtype=torch.int64)
    final_clusters = torch.zeros(len(vecs), device=device, dtype=torch.int64) - 1
    current_cluster = 0
    M = vecs.clone()
    
    while len(M) > 0:
        which = torch.randint(low=0, high=len(M),size=(1,)).to(device)
        closest_indices = cosine_choice(which, min(MAXSIZE, len(M)), M).to(device)
        anchor = (which==closest_indices).nonzero()[0].to(device)
        closest_vectors = vecs[closest_indices][None, :, :].to(device)
        
        m = dacmodel.max_class
        dacmodel.max_class = 1
        res = dacmodel.cluster_anchored(closest_vectors, given_idxs=anchor).squeeze(0)
        dacmodel.max_class = m

        where_ones = res.nonzero().squeeze(1)
        M_indices = closest_indices[where_ones]
        cluster_indices = allvecs_indices[M_indices]
        final_clusters[cluster_indices] = current_cluster

        ones = torch.ones(len(M), dtype=torch.int64, device=device)
        ones[M_indices] = 0
        nz = ones.nonzero().squeeze(1)
        M = M[nz]
        allvecs_indices = allvecs_indices[nz]

        current_cluster += 1

    cnt = final_clusters.max().cpu().item()
    score = compute_ab(final_clusters.cpu().numpy())
    results.append(score)
    clusters.append(final_clusters.cpu().numpy())
print(results)
# print(f'{cnt} {score*100:.1f} %')

In [ ]:
amo_labels = final_clusters.cpu().numpy()
km_labels = clustered_indices_all
amo_labels.shape, km_labels.shape

from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_mutual_info_score as ami

amis = [ami(amo_c, km_labels) for amo_c in clusters]
nmis = [nmi(amo_c, km_labels) for amo_c in clusters]

In [ ]:
amis, nmis

In [ ]:
def intersection_count(ag: np.ndarray, clu: np.ndarray) -> int:
    return len(np.intersect1d(ag, clu, assume_unique=True))

vecs = vecs.to(device)

MAXSIZE=800

all_hits, all_misses = [], []
print(len(kmeans_clusters))
for i, k in enumerate(kmeans_clusters):
    for which in k:
        which = which.to(device).reshape(1)
        closest_indices = cosine_choice(which, min(MAXSIZE, len(vecs)), vecs).to(device)
        anchor = (which==closest_indices).nonzero()[0].to(device)
        closest_vectors = vecs[closest_indices][None, :, :]
        res = dacmodel.cluster_anchored(closest_vectors, given_idxs=anchor).squeeze(0)
        
        where_ones = res.nonzero().squeeze(1)
        predicted = closest_indices[where_ones].cpu().numpy()
        
        hits = intersection_count(predicted, k)
        misses = len(predicted) - hits
        all_hits.append(hits)
        all_misses.append(misses)
        # intersections: int = intersection_count(res.cpu().numpy(), k)
    print('.', end='')

In [ ]:
sum(h for h in all_hits), sum(m for m in all_misses)

Whereas in the Omniglot experiment, the bag size and number of elements in class was low (e.g., 100 and 5), here we need to cluster the set of 203,057 items.